# Minicurso Prático: Vector Database com Qdrant

Bem-vindo(a) à parte prática do nosso seminário!

Neste notebook, vamos aprender na prática como interagir com um banco de dados vetorial, o **Qdrant**. Cobriremos o ciclo completo de operações, desde a criação de uma "coleção" até a execução de buscas semânticas inteligentes.

**O que vamos fazer:**
1.  **Conectar** ao nosso banco de dados Qdrant rodando no Docker.
2.  **Preparar um modelo de IA** para transformar texto em vetores (embeddings).
3.  **Criar uma coleção** para armazenar nossos vetores.
4.  **Inserir dados** (documentos de texto sobre IA, culinária e história).
5.  **Realizar buscas** que entendem o significado, não apenas palavras-chave.
6.  **Filtrar resultados** combinando busca semântica e metadados.
7.  **Gerenciar os dados** com operações de deleção e contagem.

Vamos começar!

## Passo 1: Importando as Bibliotecas e Conectando ao Banco

Tudo começa aqui. Vamos importar as bibliotecas necessárias para o nosso trabalho e estabelecer a conexão com a instância do Qdrant que está rodando no Docker.

- `qdrant_client`: A biblioteca oficial para "conversar" com o Qdrant.
- `sentence_transformers`: A biblioteca que contém o nosso modelo de IA para gerar os vetores.

In [4]:
# Importações principais
from qdrant_client import QdrantClient, models
from sentence_transformers import SentenceTransformer
import numpy as np # É uma boa prática importar, pois é muito usado em operações com vetores

# --- Conexão com o Cliente Qdrant ---
# Se o Qdrant está rodando via `docker-compose up`, ele estará acessível neste endereço.
try:
    client = QdrantClient(host="localhost", port=6333)
    print("✅ Conexão com o Qdrant estabelecida com sucesso!")
    print("Versão do Qdrant:", client.get_collections()) # Um pequeno teste para verificar a conexão
except Exception as e:
    print(f"❌ Falha ao conectar com o Qdrant. Verifique se o contêiner Docker está rodando.")
    print(f"Erro: {e}")

✅ Conexão com o Qdrant estabelecida com sucesso!
Versão do Qdrant: collections=[]


## Passo 2: Preparando o Modelo de IA e a Coleção

Agora que estamos conectados, precisamos de duas coisas antes de inserir dados:

1.  **Um modelo de IA:** Ele será nossa "ferramenta" para transformar textos (que o banco não entende) em vetores numéricos (que o banco entende). Usaremos um modelo pré-treinado da biblioteca `Sentence Transformers`.
2.  **Uma "coleção" no Qdrant:** É o espaço, similar a uma tabela, onde nossos vetores e metadados serão armazenados.

Vamos carregar o modelo e definir as configurações da nossa coleção.

In [5]:
# Carrega o modelo pré-treinado 'all-MiniLM-L6-v2' da biblioteca SentenceTransformer.
# Este modelo é leve e eficiente, ótimo para demonstrações. Ele é o responsável
# por converter nossas frases em vetores.
model = SentenceTransformer('all-MiniLM-L6-v2')

# Precisamos saber o tamanho (a dimensionalidade) dos vetores que o modelo gera.
# Para este modelo, o tamanho é 384.
vector_size = model.get_sentence_embedding_dimension()

# Por fim, definimos o nome da nossa coleção.
collection_name = "documentos_ia"

print(f"Modelo '{model.__class__.__name__}' carregado.")
print(f"Tamanho do vetor (dimensionalidade): {vector_size}")
print(f"Nome da coleção que será criada: '{collection_name}'")

Modelo 'SentenceTransformer' carregado.
Tamanho do vetor (dimensionalidade): 384
Nome da coleção que será criada: 'documentos_ia'


Com as configurações prontas, vamos efetivamente criar a coleção no Qdrant.

**Importante:** O código abaixo primeiro tenta deletar a coleção se ela já existir. Fazemos isso para que, durante o minicurso, possamos rodar o notebook várias vezes desde o início sem receber um erro de "coleção já existe". É uma prática comum em notebooks de demonstração para garantir um ambiente limpo a cada execução.

In [6]:
# Bloco `try/except` para deletar a coleção se ela já existir
try:
    client.delete_collection(collection_name=collection_name)
    print(f"ℹ️ Coleção '{collection_name}' antiga encontrada e deletada para garantir um início limpo.")
except Exception as e:
    print(f"ℹ️ Coleção '{collection_name}' não existia previamente. Tudo certo para continuar.")

# Cria a nova coleção com a configuração de vetores que definimos
client.create_collection(
    collection_name=collection_name,
    vectors_config=models.VectorParams(
        size=vector_size,      # O tamanho do vetor (384)
        distance=models.Distance.COSINE  # A métrica de similaridade de cosseno
    ),
)

print(f"✅ Coleção '{collection_name}' criada com sucesso no Qdrant!")

ℹ️ Coleção 'documentos_ia' antiga encontrada e deletada para garantir um início limpo.
✅ Coleção 'documentos_ia' criada com sucesso no Qdrant!


## Passo 3: Inserindo Dados na Coleção

Com a nossa coleção vazia criada, esta é a etapa de 'escrita' no banco de dados. Vamos pegar uma lista de frases simples, com temas variados, e prepará-las para serem inseridas.

Cada 'item' que inserimos no Qdrant é chamado de **ponto** (Point). Um `ponto` é a estrutura fundamental e é composto por 3 partes principais:
- `id`: Um identificador único para o ponto (como uma chave primária).
- `vector`: O vetor numérico gerado pelo nosso modelo de IA. É a representação matemática do dado.
- `payload`: Um dicionário com dados extras que queremos associar ao vetor (metadados), como o texto original, categorias, datas, etc.

In [9]:
# Primeiro, definimos a nossa lista de documentos em um formato de dicionário Python.
# Cada dicionário contém o texto a ser vetorizado e uma categoria como metadado.
documents = [
    {"text": "A inteligência artificial está revolucionando a medicina.", "category": "tecnologia"},
    {"text": "Grandes modelos de linguagem (LLMs) são a base de muitos chatbots.", "category": "tecnologia"},
    {"text": "O aprendizado de máquina é um subcampo da IA.", "category": "tecnologia"},
    {"text": "A culinária italiana é famosa por suas massas e pizzas.", "category": "culinaria"},
    {"text": "Receitas saudáveis com vegetais frescos e orgânicos.", "category": "culinaria"},
    {"text": "Explorando a história da Roma Antiga e seus imperadores.", "category": "historia"},
    {"text": "Tutankhamon foi um faraó egípcio.", "category": "historia"},
]

print(f"Temos {len(documents)} documentos de exemplo para inserir.")

Temos 7 documentos de exemplo para inserir.


Agora, vamos iterar (fazer um loop) sobre cada documento da lista acima. Para cada um, faremos duas coisas:

1.  **Gerar o Embedding:** Usar nosso modelo (`model`) para transformar o texto em um vetor de 384 dimensões.
2.  **Criar o Ponto:** Montar a estrutura do `PointStruct` do Qdrant com o ID, o vetor gerado e o payload (nossos metadados).

In [10]:
# Inicializamos uma lista vazia para guardar nossos pontos
points = []

# Loop para percorrer os documentos e transformá-los em pontos
for idx, doc in enumerate(documents):
    # Gerar o embedding: pega o texto e o transforma em um vetor numérico.
    embedding = model.encode(doc["text"]).tolist()
    
    # Cria a estrutura do ponto com ID, vetor e metadados (payload).
    points.append(
        models.PointStruct(
            id=idx,  # Usamos o índice do loop como um ID único
            vector=embedding,
            payload={
                "original_text": doc["text"],
                "category": doc["category"]
            }
        )
    )

# Ao final do loop, a lista 'points' conterá todos os nossos dados prontos para serem enviados.
print(f"Estrutura de dados para {len(points)} pontos foi criada com sucesso.")
print("\nExemplo do primeiro ponto a ser inserido (para fins de visualização):")
print(points[0])

Estrutura de dados para 7 pontos foi criada com sucesso.

Exemplo do primeiro ponto a ser inserido (para fins de visualização):
id=0 vector=[-0.02474880963563919, 0.008848239667713642, -0.019703147932887077, -0.027849934995174408, -0.11648442596197128, -0.020190618932247162, 0.006632782053202391, 0.062217485159635544, 0.036311790347099304, 0.0348927266895771, 0.08233705163002014, 0.052650146186351776, -0.046630240976810455, -0.03433861956000328, -0.03502022475004196, 0.022223206236958504, 0.01072789542376995, 0.011419301852583885, 0.00830154214054346, 0.05925355851650238, 0.005376025568693876, -0.040512777864933014, -0.06499379873275757, 0.06454021483659744, -0.0726976990699768, 0.04193728417158127, 0.03962210193276405, -0.03631976246833801, 0.005612739361822605, -0.09428752958774567, 0.10018959641456604, 0.07958005368709564, 0.08315685391426086, -0.08576337993144989, -0.013789718970656395, -0.005622575059533119, 0.058065664023160934, -0.04129907488822937, 0.06081412360072136, 0.089695

Com nossa lista de pontos pronta, usamos um único comando, `upsert`, para enviar todos eles para o Qdrant de uma vez. O comando `upsert` é eficiente pois ele insere novos pontos ou atualiza os que já existem (caso o ID já esteja no banco).

In [11]:
# Envia a lista de pontos para a nossa coleção no Qdrant.
operation_info = client.upsert(
    collection_name=collection_name,
    wait=True,  # Pede para a operação esperar a conclusão antes de continuar
    points=points,
)

print("✅ Dados inseridos no Qdrant!")
print("\nInformação da operação retornada pelo servidor:")
print(operation_info)

✅ Dados inseridos no Qdrant!

Informação da operação retornada pelo servidor:
operation_id=0 status=<UpdateStatus.COMPLETED: 'completed'>


## Passo 4: Realizando Buscas Semânticas (CRUD - Read)

Com nossos dados no banco, é hora de consultá-los. Diferente de uma busca tradicional em SQL com `WHERE texto LIKE '%palavra%'`, uma busca vetorial encontra os resultados mais "próximos" em significado.

O processo é simples:
1.  Definimos uma frase de busca (nossa "pergunta").
2.  Usamos o **mesmo modelo de IA** para converter essa frase em um vetor.
3.  Enviamos esse vetor para o Qdrant.
4.  O Qdrant calcula a "distância" (usando a métrica de cosseno que definimos) entre o nosso vetor de busca e todos os vetores da coleção e nos retorna os mais próximos, ou seja, os mais similares.

### 4.1. Exemplo 1: Busca Semântica Pura

Vamos começar com uma busca sobre **"modelos de IA para conversação"**. Note que a palavra "chatbot" não está na nossa frase de busca, mas está em um dos documentos que inserimos. Vamos ver se o Qdrant é inteligente o suficiente para entender essa conexão semântica.

In [12]:
# 1. Definimos a nossa busca
query_text_1 = "modelos de IA para conversação"

# 2. Convertemos a busca em um vetor
query_embedding_1 = model.encode(query_text_1).tolist()

# 3. Executamos a busca no Qdrant
search_result_1 = client.search(
    collection_name=collection_name,
    query_vector=query_embedding_1,
    limit=3,  # Pedimos os 3 resultados mais próximos
    with_payload=True  # Dizemos para incluir os metadados (nosso texto original e categoria)
)

# 4. Exibimos os resultados de forma legível
print(f"--- Resultados da busca por: '{query_text_1}' ---")
for hit in search_result_1:
    print(f"\nID do Ponto: {hit.id}")
    print(f"Similaridade (Score): {hit.score:.4f}")
    print(f"  Texto Original: {hit.payload['original_text']}")
    print(f"  Categoria: {hit.payload['category']}")

--- Resultados da busca por: 'modelos de IA para conversação' ---

ID do Ponto: 1
Similaridade (Score): 0.4922
  Texto Original: Grandes modelos de linguagem (LLMs) são a base de muitos chatbots.
  Categoria: tecnologia

ID do Ponto: 2
Similaridade (Score): 0.4148
  Texto Original: O aprendizado de máquina é um subcampo da IA.
  Categoria: tecnologia

ID do Ponto: 0
Similaridade (Score): 0.4068
  Texto Original: A inteligência artificial está revolucionando a medicina.
  Categoria: tecnologia


C:\Users\55819\AppData\Local\Temp\ipykernel_101664\3154319202.py:8: DeprecationWarning: `search` method is deprecated and will be removed in the future. Use `query_points` instead.
  search_result_1 = client.search(


### 4.2. Exemplo 2: Combinando Busca Semântica com Filtros

Esta é uma das funcionalidades mais úteis dos bancos de dados vetoriais modernos. E se quisermos encontrar textos sobre um assunto, mas apenas dentro de uma **categoria específica**?

Vamos buscar por **"comida boa"**, mas restringindo a busca **apenas** para os documentos da categoria `culinaria`. Isso evita que o resultado traga, por exemplo, um texto sobre a "boa" estratégia de um imperador romano.

In [13]:
# 1. Definimos a nova busca
query_text_2 = "comida boa"

# 2. Convertemos em um vetor
query_embedding_2 = model.encode(query_text_2).tolist()

# 3. Executamos a busca, mas desta vez adicionando um filtro
search_result_2 = client.search(
    collection_name=collection_name,
    query_vector=query_embedding_2,
    # AQUI ESTÁ O FILTRO: definimos uma condição obrigatória (must)
    # onde o campo 'category' no payload deve ter o valor 'culinaria'.
    query_filter=models.Filter(
        must=[
            models.FieldCondition(
                key="category",
                match=models.MatchValue(value="culinaria")
            )
        ]
    ),
    limit=2, # Pedimos os 2 melhores resultados dentro do filtro
    with_payload=True
)

# 4. Exibimos os resultados
print(f"--- Resultados da busca por: '{query_text_2}' (filtrado por Categoria: culinaria) ---")
for hit in search_result_2:
    print(f"\nID do Ponto: {hit.id}")
    print(f"Similaridade (Score): {hit.score:.4f}")
    print(f"  Texto Original: {hit.payload['original_text']}")
    print(f"  Categoria: {hit.payload['category']}")

--- Resultados da busca por: 'comida boa' (filtrado por Categoria: culinaria) ---

ID do Ponto: 4
Similaridade (Score): 0.2985
  Texto Original: Receitas saudáveis com vegetais frescos e orgânicos.
  Categoria: culinaria

ID do Ponto: 3
Similaridade (Score): 0.2749
  Texto Original: A culinária italiana é famosa por suas massas e pizzas.
  Categoria: culinaria


C:\Users\55819\AppData\Local\Temp\ipykernel_101664\1328935485.py:8: DeprecationWarning: `search` method is deprecated and will be removed in the future. Use `query_points` instead.
  search_result_2 = client.search(


## Passo 5: Gerenciando os Dados (Retrieve, Delete e Count)

Além de buscar por similaridade, que é a função principal, muitas vezes precisamos realizar operações diretas nos dados. Por exemplo:
- Buscar um item que já conhecemos pelo seu ID.
- Remover dados que não são mais necessários ou estão desatualizados.
- Verificar quantos itens temos no total.

### 5.1. Buscando um Ponto Específico por ID (`retrieve`)

Imagine que você já sabe o ID de um documento e quer apenas os dados dele, sem fazer uma busca por similaridade. Para isso, usamos o comando `retrieve`, que é extremamente rápido.

In [14]:
# Vamos buscar o ponto que tem o ID 2 (no nosso caso, sobre "aprendizado de máquina")
point_id_to_retrieve = 2

retrieved_points = client.retrieve(
    collection_name=collection_name,
    ids=[point_id_to_retrieve],
    with_payload=True # Pedimos para incluir os metadados
)

print(f"--- Buscando o ponto com ID: {point_id_to_retrieve} ---")
print("Resultado:")
print(retrieved_points)

--- Buscando o ponto com ID: 2 ---
Resultado:
[Record(id=2, payload={'original_text': 'O aprendizado de máquina é um subcampo da IA.', 'category': 'tecnologia'}, vector=None, shard_key=None, order_value=None)]


### 5.2. Deletando Pontos da Coleção (`delete`)

Agora, vamos simular a remoção de alguns dados. A operação `delete` permite remover um ou mais pontos de forma eficiente, bastando fornecer seus IDs. Vamos deletar os pontos com ID 3 e 4 (os de culinária).

In [15]:
# Deleta os pontos com IDs 3 e 4 da nossa coleção.
operation_info_delete = client.delete(
    collection_name=collection_name,
    points_selector=models.PointIdsList(points=[3, 4])
)

print("--- Deletando os pontos com ID 3 e 4 ---")
print("Operação de deleção concluída!")
print(operation_info_delete)

--- Deletando os pontos com ID 3 e 4 ---
Operação de deleção concluída!
operation_id=1 status=<UpdateStatus.COMPLETED: 'completed'>


### 5.3. Verificando o Estado Final da Coleção (`count`)

Depois de deletar, como podemos confirmar que a operação funcionou? Usamos o comando `count` para ver quantos pontos restaram na coleção. Começamos com 7, deletamos 2, então o resultado deve ser 5.

In [16]:
# Conta o número de pontos restantes na coleção
count_result = client.count(
    collection_name=collection_name,
    exact=True # Pede uma contagem exata
)

print("--- Contando o total de pontos restantes ---")
print(f"Total de pontos na coleção '{collection_name}': {count_result.count}")

--- Contando o total de pontos restantes ---
Total de pontos na coleção 'documentos_ia': 5


## Conclusão do Notebook

Você completou o ciclo de vida de interação com um banco de dados vetorial. Agora você sabe como:
- **Configurar e conectar** a um banco de dados vetorial.
- **Criar uma coleção** para armazenar vetores.
- **Inserir dados** usando embeddings.
- **Realizar buscas** semânticas e com filtros.
- **Gerenciar os dados** com operações de busca por ID, deleção e contagem.

Com este conhecimento, você está pronto para o próximo e último passo do nosso projeto!

### Próximo Passo do Projeto Geral

Usar todo esse conhecimento para construir o **caso de uso final**: uma aplicação web interativa com Streamlit, que será o arquivo `src/app.py`. Lá, vamos focar em usar a operação de `search` para criar uma experiência de busca incrível para o usuário.

Teste no terminal: **streamlit run src/app.py**